In [62]:
import cv2 
import numpy as np

feature_params =  dict(maxCorners = 300,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 1)
lk_params = dict(winSize = (15, 15),
                 maxLevel = 3,
                 criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

#preluare video
cap = cv2.VideoCapture('Train.mp4')
ret, prev = cap.read()
fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
out = cv2.VideoWriter('output.avi', fourcc, 29.0, (1280, 720))
prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(prevgray, mask = None, **feature_params)
mask = np.zeros_like(prev)
transform = []
while (cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        p1, st, err = cv2.calcOpticalFlowPyrLK(prevgray, gray, p0, None, **lk_params)
        
        good_new = p1[st == 1]
        good_old = p0[st == 1]
        
        matrix, _ = cv2.estimateAffine2D(good_old, good_new)
        
        if matrix is not None:
            dx = matrix[0, 2]
            dy = matrix[1, 2]
            da = np.arctan2(matrix[1,0], matrix[0,0])
            transform.append([dx, dy, da])
        
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            # mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)
            frame = cv2.circle(frame, (int(a), int(b)), 5, (0, 255, 0), -1)
        img = cv2.add(frame, mask)
        
        prevgray = gray.copy()
        p0 = good_new.reshape(-1, 1, 2)
        
        
        out.write(img)
    else:
        break 
transform = np.array(transform)

trajectory = np.cumsum(transform, axis=0)
smoothed_trajectory = np.convolve(trajectory[:, 0], np.ones(w)/w, mode='valid')

#window size
w = 30

dx_smooth = np.interp(np.arange(len(trajectory[:, 0])), np.arange(len(smoothed_trajectory)), smoothed_trajectory) 
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

for i, (dx, dy, da) in enumerate(transform):
    ret, frame = cap.read()
    if not ret:
        break
    smoothed_matrix = np.array([[np.cos(da), -np.sin(da), dx],
                                [np.sin(da), np.cos(da), dy]])
    
    stabilized_frame = cv2.warpAffine(frame, smoothed_matrix, (frame.shape[1], frame.shape[0]))
    
    height, width = stabilized_frame.shape[:2]
    crop_margin = 20
    stabilized_frame = stabilized_frame[crop_margin:height-crop_margin, crop_margin:width-crop_margin]
    stabilized_frame = cv2.resize(stabilized_frame, (width,height))
    out.write(stabilized_frame)
out.release()
cap.release()
cv2.destroyAllWindows()


In [66]:
# Parameters for ShiTomasi corner detection
feature_params = dict(maxCorners=200, qualityLevel=0.01, minDistance=30, blockSize=3)

# Parameters for Lucas-Kanade optical flow
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Smoothing function
def smooth_trajectory(trajectory, smooth_radius=30):
    smoothed = np.zeros_like(trajectory)
    for i in range(trajectory.shape[1]):
        smoothed[:, i] = np.convolve(trajectory[:, i], np.ones(smooth_radius) / smooth_radius, mode='same')
    return smoothed

# Open video
cap = cv2.VideoCapture('Train2.mp4')
if not cap.isOpened():
    print("Error: Cannot open video.")
    exit()

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
out = cv2.VideoWriter('stabilized_output.avi', fourcc, fps, (width, height))

# Read the first frame
ret, prev_frame = cap.read()
if not ret:
    print("Error: Cannot read the first frame.")
    cap.release()
    exit()

prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
prev_points = cv2.goodFeaturesToTrack(prev_gray, mask=None, **feature_params)

# Arrays to store transformations
transforms = []

while True:
    ret, curr_frame = cap.read()
    if not ret:
        break

    curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow
    curr_points, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_points, None, **lk_params)

    # Filter valid points
    valid_prev_points = prev_points[status == 1]
    valid_curr_points = curr_points[status == 1]

    # Estimate transformation matrix
    matrix, _ = cv2.estimateAffinePartial2D(valid_prev_points, valid_curr_points)
    
    if matrix is not None:
        dx = matrix[0, 2]
        dy = matrix[1, 2]
        da = np.arctan2(matrix[1, 0], matrix[0, 0])
        transforms.append([dx, dy, da])
    else:
        transforms.append([0, 0, 0])

    # Update previous frame and points
    prev_gray = curr_gray.copy()
    prev_points = valid_curr_points.reshape(-1, 1, 2)

# Convert transformations to a NumPy array
transforms = np.array(transforms)

# Smooth the trajectory
trajectory = np.cumsum(transforms, axis=0)
smoothed_trajectory = smooth_trajectory(trajectory)
smoothing_transform = smoothed_trajectory - trajectory
corrected_transforms = transforms + smoothing_transform

# Apply the smoothed transformations to stabilize the video
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
for i, (dx, dy, da) in enumerate(corrected_transforms):
    ret, frame = cap.read()
    if not ret:
        break

    # Build the transformation matrix
    transformation_matrix = np.array([
        [np.cos(da), -np.sin(da), dx],
        [np.sin(da), np.cos(da), dy]
    ])

    # Apply the transformation
    stabilized_frame = cv2.warpAffine(frame, transformation_matrix, (width, height))

    # Crop to remove border artifacts
    crop_margin = 20
    stabilized_frame = stabilized_frame[crop_margin:height - crop_margin, crop_margin:width - crop_margin]
    stabilized_frame = cv2.resize(stabilized_frame, (width, height))

    # Write the stabilized frame to the output video
    out.write(stabilized_frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()